In [1]:
# !pip install opencv-python
import cv2

In [2]:
# ### Take picture from webcam and save it ##########
# For Camera Enable

#import time
# import cv2
# video_port = 0
# video = cv2.VideoCapture (video_port)
# while True:
#     ret , video_data = video.read()
#     cv2.imshow("video_live",video_data)
#     if cv2.waitKey(10) == ord('a'):
#         break
# video.release()

In [4]:
## Go to this location for importing cv2 face data "C:/ProgramData/anaconda3/Lib/site-packages/cv2/data"

face_cap = cv2.CascadeClassifier("C:/ProgramData/anaconda3/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml")
video_port = 0
video_cap = cv2.VideoCapture (video_port)
while True:
    ret , video_data = video_cap.read()
    col = cv2.cvtColor(video_data,cv2.COLOR_BGR2GRAY) # for making face color to black n white for getting face muscle properly. then again make color.
    faces= face_cap.detectMultiScale(col, scaleFactor= 1.1, minNeighbors= 5, minSize=(30,30), flags = cv2.CASCADE_SCALE_IMAGE )
    
    for (x,y,w,h) in faces:
        cv2.rectangle(video_data,(x,y),(x+w, y+h), (0,255,0), 2)
    cv2.imshow("video_live",video_data)
    if cv2.waitKey(10) == ord('a'):
        break
video_cap.release()

# Face recognition using Deep Learning(CNN) 

1. Convolutional neural networks (CNN) is used to convert digital image content into a single vector of numbers(numeric vector) representing the unique characteristics of the image.

2. The column of numbers is inputted to a Dense fully connected Neural network layer against the labels, which image is cat, which image is bird etc.

3. The classification model learns these numeric vector inputs against the labels


### Reading the Images data

In [1]:
 # Deep Learning CNN model to recognize face
'''This script uses a database of images and creates CNN model on top of it to test
if the given image is recognized correctly or not'''

'''######### IMAGE PRE-PROCESSING for TRAINING and TESTING data ########'''

TrainingImagePath='D:/Coding Course/IVY Pro School/ML/Face Images/Final Training Images'

from keras.preprocessing.image import ImageDataGenerator
#Understand more about ImageDataGenerator at below link
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-di

# Defining pre-processing transformations on raw images of training data 
train_datagen = ImageDataGenerator (
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)
# Defining pre-processing transformations on raw images of testing data
test_datagen = ImageDataGenerator(rescale=1./255)

#Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')



#Generating the Testing Data
test_set = test_datagen.flow_from_directory(
    TrainingImagePath,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical')

# Printing class labels for each face
test_set.class_indices



Found 244 images belonging to 16 classes.
Found 244 images belonging to 16 classes.


{'face1': 0,
 'face10': 1,
 'face11': 2,
 'face12': 3,
 'face13': 4,
 'face14': 5,
 'face15': 6,
 'face16': 7,
 'face2': 8,
 'face3': 9,
 'face4': 10,
 'face5': 11,
 'face6': 12,
 'face7': 13,
 'face8': 14,
 'face9': 15}

### Creating a list of faces and classes

In [2]:
'''###### Creating lookup table for all faces ####'''
# class_indices have the numeric tag for each face 
TrainClasses = training_set.class_indices

#Storing the face and the numeric tag for future reference
ResultMap={}
for faceValue, faceName in zip (TrainClasses.values(), TrainClasses.keys()): 
    ResultMap[faceValue]=faceName

    # Saving the face map for future reference
import pickle
with open("ResultsMap.pkl", 'wb') as f:
    pickle.dump (ResultMap, f, pickle.HIGHEST_PROTOCOL)

print("Mapping of Face and its ID", ResultMap)

# The number of neurons for the output layer is equal to the number of faces OutputNeurons-len (ResultMap)
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons:', OutputNeurons)


Mapping of Face and its ID {0: 'face1', 1: 'face10', 2: 'face11', 3: 'face12', 4: 'face13', 5: 'face14', 6: 'face15', 7: 'face16', 8: 'face2', 9: 'face3', 10: 'face4', 11: 'face5', 12: 'face6', 13: 'face7', 14: 'face8', 15: 'face9'}

 The Number of output neurons: 16


### Creating the CNN model

In [3]:
#####
########## Create CNN deep learning model #####
####*


from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
 
'''Initializing the Convolutional Neural Network'''
classifier= Sequential()
 
''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (64,64,3) because we are using TensorFlow backend
# It means 3 matrix of size (64X64) pixels representing Red, Green and Blue components of pixels
'''
classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))
 
'''# STEP--2 MAX Pooling'''
classifier.add(MaxPool2D(pool_size=(2,2)))
 
'''############## ADDITIONAL LAYER of CONVOLUTION for better accuracy #################'''
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
 
classifier.add(MaxPool2D(pool_size=(2,2)))
 
'''# STEP--3 FLattening'''
classifier.add(Flatten())
 
'''# STEP--4 Fully Connected Neural Network'''
classifier.add(Dense(64, activation='relu'))
 
classifier.add(Dense(OutputNeurons, activation='softmax'))
 
'''# Compiling the CNN'''
#classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])
 


## Fitting the model

In [4]:
import time

# Measuring the time taken by the model to train
StartTime = time.time()

# Manually repeat the generator for a specified number of epochs
steps_per_epoch = len(training_set)

# Starting the model training
classifier.fit(
    training_set,
    epochs=30,
    validation_data=test_set,
    steps_per_epoch=steps_per_epoch,
    validation_steps=10
)

EndTime = time.time()
print("###### Total Time Taken: ", round((EndTime - StartTime) / 60), 'Minutes ######')


Epoch 1/30


8/8 [==============================] - 14s 1s/step - loss: 2.8658 - accuracy: 0.0779 - val_loss: 2.7610 - val_accuracy: 0.1066
Epoch 2/30
8/8 [==============================] - 5s 521ms/step - loss: 2.7537 - accuracy: 0.1107
Epoch 3/30
8/8 [==============================] - 4s 446ms/step - loss: 2.7029 - accuracy: 0.1516
Epoch 4/30
8/8 [==============================] - 3s 369ms/step - loss: 2.5639 - accuracy: 0.2336
Epoch 5/30
8/8 [==============================] - 3s 345ms/step - loss: 2.4236 - accuracy: 0.3320
Epoch 6/30
8/8 [==============================] - 3s 346ms/step - loss: 2.1706 - accuracy: 0.4385
Epoch 7/30
8/8 [==============================] - 3s 344ms/step - loss: 1.7998 - accuracy: 0.5123
Epoch 8/30
8/8 [==============================] - 3s 345ms/step - loss: 1.3792 - accuracy: 0.6516
Epoch 9/30
8/8 [==============================] - 3s 347ms/step - loss: 1.0239 - accuracy: 0.7377
Epoch 10/30
8/8 [==============================] - 3s 363ms/step - loss: 0.7

## Testing the CNN classifier on unseen images

Using any one of the images from the testing data folder, we can check if the model is able to recognize the face.

In [7]:
'''########### Making single predictions ###########'''
import numpy as np
from keras.preprocessing import image
 
ImagePath='D:/Coding Course/IVY Pro School/ML/Face Images/Final Testing Images/face3/3face3.jpg'
test_image=image.load_img(ImagePath,target_size=(64, 64))
test_image=image.img_to_array(test_image)
 
test_image=np.expand_dims(test_image,axis=0)
 
result=classifier.predict(test_image,verbose=0)
print(result)
print(ResultMap)

#print(training_set.class_indices)
 
print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
{0: 'face1', 1: 'face10', 2: 'face11', 3: 'face12', 4: 'face13', 5: 'face14', 6: 'face15', 7: 'face16', 8: 'face2', 9: 'face3', 10: 'face4', 11: 'face5', 12: 'face6', 13: 'face7', 14: 'face8', 15: 'face9'}
########################################
Prediction is:  face3
